# Agent Based Model: Main Notebook

In [471]:
import numpy.random as rnd
import numpy as np
import pandas as pd 
from matplotlib import pyplot as plt
import random

#### Workflow Notes 
* Draw Students with values within the four dimensions 
* Function that creates a study group (draw 4 students) -> Measure homogeniety 
* Task 


## Importing The Study Groups that are to be Tested

In [472]:
all_groups = pd.read_csv("pt_data.csv")


## Creating Study Groups

#### Practical Functions

In [473]:
def n_sampler(size):
    '''
    sample a number of random numbers from the beta distribution
    '''
    max_vals = []
    min_vals = []

    beta_dist = rnd.beta(1.5, 1.5, size)

    for i in range(size):
        if beta_dist[i] >= 0.5:
            max_vals.append(beta_dist[i])
        else:
            min_vals.append(beta_dist[i])
    
    return max_vals, min_vals

max_vals, min_vals = n_sampler(1000000)

In [474]:
def data_collect(studygroup, student_list):
    '''
    function to collect data from the simulation
    '''
    Name_list = []
    extraversion_list = []
    sensing_list = []
    thinking_list = []
    judging_list = []
    academic_list = []
    
    for student in studygroup:
        Name_list.append(student.Name)
        extraversion_list.append(student.ExScore)
        sensing_list.append(student.SeScore) 
        thinking_list.append(student.ThScore)
        judging_list.append(student.JuScore)
        academic_list.append(student.Academic_Skill)

    data = pd.DataFrame({'Name': Name_list, 
                        'type': student_list, 
                        'E/I': extraversion_list, 
                        'S/N': sensing_list,
                        'T/F': thinking_list,
                        'J/P': judging_list, 
                        'Academic': academic_list})
    
    return data
    

#### Create Study Group

In [475]:
## Defining the Students ##
class Student():
    def __init__(self, Name, Ex, Se, Th, Ju):
        self.Name  = Name

        ## Personality Traits ##
        self.Ex = Ex #Extraversion vs Introversion dimension
        self.Se = Se #Sensing vs Intuition dimension
        self.Th = Th #Thinking vs Feeling dimension
        self.Ju = Ju #Judging vs Perceiving dimension
        self.Type = Ex + Se + Th + Ju

        ## Personality Scores calculated with the personality() function##
        self.ExScore = 0
        self.SeScore = 0
        self.ThScore = 0
        self.JuScore = 0
        
        self.Scores = [] #list of all personality scores

        ## Academic Skills ##
        self.Academic_Skill = 0

        ## Own Solution ##
        self.Ind_Solution = []

def personality(student):
    # Extraversion vs. Introversion
    if student.Ex == "E":
        student.ExScore = random.choice(max_vals)
    else:
        student.ExScore = random.choice(min_vals)
    
    # Sensing vs. Intuition
    if student.Se == "S":
        student.SeScore = random.choice(max_vals)
    else:
        student.SeScore = random.choice(min_vals)
    
    # Thinking vs. Feeling
    if student.Th == "T":
        student.ThScore = random.choice(max_vals)
    else:
        student.ThScore = random.choice(min_vals)

    # Judging vs. Perceiving
    if student.Ju == "J":
        student.JuScore = random.choice(max_vals)
    else:
        student.JuScore = random.choice(min_vals)
    
    student.Scores = [student.ExScore, student.SeScore, student.ThScore, student.JuScore]

def skills(student):
    student.Academic_Skill = (1-student.SeScore)*0.33 + student.JuScore*0.33 + (rnd.beta(8, 2, 1)[0]*0.33)
    student.Compromising = (1-student.ThScore)*0.33 + student.ExScore*0.33 + (rnd.beta(8, 2, 1)[0]*0.33)

In [476]:
def StudyGroup(student_list):
    '''
    Create a study group of students
    '''
    studygroup = []
    names = ["Alfa", "Bravo", "Charlie", "Delta"]

    for i in range(len(student_list)):
        student = Student(names[i], student_list[i][0], student_list[i][1], student_list[i][2], student_list[i][3])
        personality(student)
        skills(student)
        studygroup.append(student)
    
    return data_collect(studygroup, student_list), studygroup

## Creating the ABM

### Generating True Solution

In [477]:
def true_solution_generator(n_part_exercises, range_elements):
    '''
    create a list of random numbers that will serve as the true solution that the agents need to find 
    '''
    true_solution = []
    for i in range(n_part_exercises):
        true_solution.append(random.randint(range_elements[0], range_elements[1]))

    return true_solution, range_elements

### Generating Individual Solutions

In [478]:
def individual_solutions_generator(studygroup, true_solution, range_elements):
    '''
    #function to calculate the individual solutions of the agents given a study group dataframe and a true solution
    '''
    
    all_solutions = []

    for student in studygroup[1]: # loop for each individual
        Ind_Solution_lst = []
        Ind_Solution_attr_lst = []
        
        for i in range(len(true_solution)): # loop for each part-exercise
            coin_toss = np.random.binomial(1, (student.Academic_Skill*0.5) + 0.5, 1)[0] # biased-coin flip
            if coin_toss == 1:
                Ind_Solution_lst.append(true_solution[i])
                Ind_Solution_attr_lst.append(true_solution[i])
            else:
                wrong_answer = random.randint(range_elements[0], range_elements[1])
                Ind_Solution_lst.append(wrong_answer)
                Ind_Solution_attr_lst.append(wrong_answer)
        all_solutions.append(Ind_Solution_lst)
        student.Ind_Solution = Ind_Solution_attr_lst
    
    return all_solutions

In [479]:
def agent_type_equality(agent_a, agent_b):
    equality_score = 0 # number from 0 to 1
    for i in range(4):
        if agent_a[i] == agent_b[i]:
            equality_score += 0.25
    return equality_score 

In [480]:
A = [1, 3, 2]
B = [1, 3, 2]
C = [1, 3, 3]

for i in range(3):
    if A[i] == B[i]:
        C[i] = A[i]
    if A[i] == C[i]:
        B[i] = A[i]
    if B[i] == C[i]:
        A[i] = B[i]


print(A)
print(B)
print(C) 


[1, 3, 2]
[1, 3, 2]
[1, 3, 2]


## Collaborative Problem Solving

##### Workflow notes
1. Who presents their solution e.g. agent A presents their solution to agent B, C, D -> THE PROPOSED SOLUTION
    -> Based on Extraversion score (Highest extraversion score is the most likely to present their solution)
2. According to an Agreeableness score (Social score for now) of the other agents (and maybe a Trustworthiness score of agent proposing), agents will update their solution
3. The solutions of the agents will be checked, if all they agree, this is their final solution. If not, the process will be repeated for a max of X ticks. If the groups do not converge, an accuracy score will still be calculated. 

In [481]:
def collaborative_solution(studygroup, max_ticks):
    max_ticks = max_ticks #turns in the simulation
    n_ticks = 0
    consensus = False
    
    # extracting all 4 students
    Alfa = studygroup[1][0]
    Bravo = studygroup[1][1]
    Charlie = studygroup[1][2]
    Delta = studygroup[1][3]

    student_list = [Alfa, Bravo, Charlie, Delta]
    individual_solutions = [Alfa.Ind_Solution, Bravo.Ind_Solution, Charlie.Ind_Solution, Delta.Ind_Solution]

    for i in range(8): # loop that agrees on a solution if three of the members have the same solution before we start talking.
            if (individual_solutions[0][i] == individual_solutions[1][i] == individual_solutions[2][i]): # delta is odd one out
                individual_solutions[3][i] = individual_solutions[0][i]
            
            if (individual_solutions[0][i] == individual_solutions[1][i] == individual_solutions[3][i]): # charlie is odd one out
                individual_solutions[2][i] = individual_solutions[0][i]
            
            if (individual_solutions[0][i] == individual_solutions[2][i] == individual_solutions[3][i]): # bravo is odd one out
                individual_solutions[1][i] = individual_solutions[0][i]
            
            if (individual_solutions[1][i] == individual_solutions[2][i] == individual_solutions[3][i]): # alfa is odd one out
                individual_solutions[0][i] = individual_solutions[1][i]

    while (n_ticks != max_ticks):
        # Starting a Round
        presenter_name = random.choices([Alfa.Name, Bravo.Name, Charlie.Name, Delta.Name], weights = [Alfa.ExScore, Bravo.ExScore, Charlie.ExScore, Delta.ExScore], k = 1)[0] # selecting the presenter of the round based on weighted random draw from extraversion scores
        
        Proposed_Solution = eval(presenter_name).Ind_Solution
        #print(n_ticks, presenter_name, Proposed_Solution)

        for student in student_list:
            if student == eval(presenter_name):
                continue
            
            similarity_score = agent_type_equality(eval(presenter_name).Type, student.Type) # evaluate how equal the two agents are in personality. This bases the coinflip
            similarity_coin_toss = np.random.binomial(1, similarity_score, 1)[0]
            
            if similarity_coin_toss == 0:
                coin_toss = np.random.binomial(1, 0.5, 1)[0] # giving a 50% chance of not ending loop if you lose on similarity score.
                if coin_toss == 1:
                    continue
                
            for i in range(len(Proposed_Solution)): # looping through all part-exercises and evaluating against proposed solution.
                coin_toss = np.random.binomial(1, (student.Compromising*0.25 + eval(presenter_name).Academic_Skill*0.25), 1)[0] #high social skill has a greater chance of accepting the proposal and if the proposer has higher academic skill.
                if coin_toss == 1:
                    student.Ind_Solution[i] = Proposed_Solution[i]
                else:
                    student.Ind_Solution[i] = student.Ind_Solution[i]

        n_ticks += 1 #adding a tick to the simulation

        if all(x==individual_solutions[0] for x in individual_solutions): # initializes a break of while-loop if consensus has been reached (i.e. all solutions are the same)
            consensus = True
            break
    
    if consensus == True:
        final_group_solution = individual_solutions[0] # take one of the solutions from the group if they are all the same
    
    if consensus == False:
        final_group_solution = Proposed_Solution



    #print("Number of iterations: " + str(n_ticks))
    return final_group_solution, n_ticks, consensus # returns their collective solution


In [482]:
def solution_evaluator(proposed_solution, true_solution):
    correct_part_exercise = 0
    wrong_part_exercise = 0

    for i in range(len(true_solution)):
        if proposed_solution[i] == true_solution[i]:
            correct_part_exercise += 1
        else:
            wrong_part_exercise += 1

    return correct_part_exercise / (correct_part_exercise + wrong_part_exercise)


In [483]:
def adaptation_degree(individual_solution, group_solution):
    '''
    #function to calculate the adaptation degree of the agents
    '''
    adapted = 0
    not_adapted = 0

    for i in range(len(individual_solution)):
        if individual_solution[i] == group_solution[i]:
            not_adapted += 1
        else:
            adapted += 1
    
    adaptation = adapted / (adapted + not_adapted)

    return adaptation

In [484]:
def synergy_calculator(individual_accuracies, group_accuracy):
    '''
    #function to calculate the gain of the agents
    '''
    avg_of_indi_accuracy = np.mean(individual_accuracies)
    max_of_indi_accuracy = np.max(individual_accuracies)
    
    weak_synergy = group_accuracy - avg_of_indi_accuracy
    strong_synergy = group_accuracy - max_of_indi_accuracy

    return weak_synergy, strong_synergy

In [485]:
def diversity_score_calculator(student_list):
    diversity_score = 0

    for i in range(4):
        if (student_list[0][i] == student_list[1][i] == student_list[2][i] == student_list[3][i]):
            diversity_score += 0
        elif (student_list[0][i] == student_list[1][i] == student_list[2][i]) or (student_list[0][i] == student_list[1][i] == student_list[3][i]) or (student_list[1][i] == student_list[2][i] == student_list[3][i]) or (student_list[0][i] == student_list[2][i] == student_list[3][i]):
            diversity_score += 1
        else:
            diversity_score += 2
    
    return diversity_score

# TEST RUN

In [486]:
# Creating the Study Group
student_list_test = ["ESFP", "ISFP", "ISFP", "ISFP"]
studygroup1 = StudyGroup(student_list_test)

In [487]:
diversity_score_calculator(student_list_test)

1

In [488]:
# Seeing their Generated Personality Scores
print(studygroup1[0])

      Name  type       E/I       S/N       T/F       J/P  Academic
0     Alfa  ESFP  0.509321  0.555311  0.349937  0.406134  0.471538
1    Bravo  ISFP  0.265921  0.653190  0.371819  0.480933  0.517853
2  Charlie  ISFP  0.328291  0.575488  0.320783  0.140129  0.449663
3    Delta  ISFP  0.385768  0.590190  0.300422  0.158048  0.438224


In [489]:
# Generating a True Solution
true_solution_test = true_solution_generator(8, [1, 8])

In [490]:
# Displaying the True Solution
true_solution_test[0]

[5, 1, 4, 6, 7, 4, 3, 6]

In [491]:
# Generating Individual Solutions
all_solutions = individual_solutions_generator(studygroup1, true_solution_test[0], true_solution_test[1])
all_solutions

[[5, 1, 3, 6, 7, 2, 5, 6],
 [5, 1, 4, 6, 7, 4, 3, 6],
 [5, 7, 4, 6, 7, 4, 3, 6],
 [5, 1, 4, 6, 7, 4, 3, 6]]

In [492]:
# Printing Individual Solutions
for student in studygroup1[1]:
    print(student.Name, student.Ind_Solution)

Alfa [5, 1, 3, 6, 7, 2, 5, 6]
Bravo [5, 1, 4, 6, 7, 4, 3, 6]
Charlie [5, 7, 4, 6, 7, 4, 3, 6]
Delta [5, 1, 4, 6, 7, 4, 3, 6]


In [493]:
# Printing Indivudal Accuracies
for student in studygroup1[1]:
    print(student.Name, solution_evaluator(student.Ind_Solution, true_solution_test[0]))

Alfa 0.625
Bravo 1.0
Charlie 0.875
Delta 1.0


In [494]:
# Creating Group Solution
group_solution = collaborative_solution(studygroup1, 100)

In [495]:
print(group_solution)
print(all_solutions)

([5, 1, 4, 6, 7, 4, 3, 6], 1, True)
[[5, 1, 3, 6, 7, 2, 5, 6], [5, 1, 4, 6, 7, 4, 3, 6], [5, 7, 4, 6, 7, 4, 3, 6], [5, 1, 4, 6, 7, 4, 3, 6]]


In [496]:
# Printing Group Accuracy
accuracy = solution_evaluator(group_solution[0], true_solution_test[0])
print(accuracy)

1.0


In [497]:
adaptation_degree(all_solutions[0], group_solution[0])

0.375

## ABM Model

In [498]:
def exercise_run(student_list: list, max_ticks: int, n_part_exercises: int, range_solution: list, n_simulations: int):
    '''
    function to run one simulation of a study group completing the exercise (from individual to group solution)
    '''

    #### ---- LISTS ---- #### 
    ## ---- group level ---- ###
    group_accuracy_list = []
    n_tick_list = []
    weak_synergy_list = []
    strong_synergy_list = []
    consensus_list = []

    ## ---- individual level ---- ###
    alfa_accuracy_list = []
    beta_accuracy_list = []
    charlie_accuracy_list = []
    delta_accuracy_list = []

    alfa_adaptation_list = []
    beta_adaptation_list = []
    charlie_adaptation_list = []
    delta_adaptation_list = []

    # ---- starting simulation ---- #
    for i in range(n_simulations):
        ## giving the study group personality values and skills ##
        studygroup = StudyGroup(student_list)
        
        ## ---- SOLUTIONS ---- ##
        ## generating the true solution ## 
        true_solution = true_solution_generator(n_part_exercises, range_solution)

        ## generating individual solutions ##
        all_solutions = individual_solutions_generator(studygroup, true_solution[0], true_solution[1])

        ## evaluating own solutions ##        
        alfa_accuracy = solution_evaluator(all_solutions[0], true_solution[0])
        beta_accuracy = solution_evaluator(all_solutions[1], true_solution[0])
        charlie_accuracy = solution_evaluator(all_solutions[2], true_solution[0])
        delta_accuracy = solution_evaluator(all_solutions[3], true_solution[0])

        all_accuracies = [alfa_accuracy, beta_accuracy, charlie_accuracy, delta_accuracy]
        
        alfa_accuracy_list.append(alfa_accuracy)
        beta_accuracy_list.append(beta_accuracy)
        charlie_accuracy_list.append(charlie_accuracy)
        delta_accuracy_list.append(delta_accuracy)

        ## generating group solution ##
        group_solution = collaborative_solution(studygroup, max_ticks)
        n_tick_list.append(group_solution[1]) # appending the number of ticks
        consensus_list.append(group_solution[2]) # appending whether a consensus was reached or not
        
        ## ---- MEASURES ---- ##
        ## calculating degree of adaptation ## 
        alfa_adaption = adaptation_degree(all_solutions[0], group_solution[0])
        beta_adaption = adaptation_degree(all_solutions[1], group_solution[0])
        charlie_adaption = adaptation_degree(all_solutions[2], group_solution[0])
        delta_adaption = adaptation_degree(all_solutions[3], group_solution[0])

        alfa_adaptation_list.append(alfa_adaption)
        beta_adaptation_list.append(beta_adaption)
        charlie_adaptation_list.append(charlie_adaption)
        delta_adaptation_list.append(delta_adaption)

        ## calculating the accuracy of the group solution ##
        group_accuracy = solution_evaluator(group_solution[0], true_solution[0])
        group_accuracy_list.append(group_accuracy) #appending the group accuracies

        # calculating the gain from the individual solutions to the group solution
        weak_synergy, strong_synergy = synergy_calculator(all_accuracies, group_accuracy)
        weak_synergy_list.append(weak_synergy)
        strong_synergy_list.append(strong_synergy)
        

    
    diversity_score = diversity_score_calculator(student_list)

    ## calculating the mean and standard deivations ##
    # ---- group level ---- #
    avg_group_accuracy = np.mean(group_accuracy_list)
    std_group_accuracy = np.std(group_accuracy_list)
    
    avg_group_n_tick = np.mean(n_tick_list)
    std_group_n_tick = np.std(n_tick_list)

    avg_consensus = np.mean(consensus_list)
    std_consensus = np.std(consensus_list)

    avg_weak_synergy  = np.mean(weak_synergy_list)
    std_weak_synergy = np.std(weak_synergy_list)

    avg_strong_synergy  = np.mean(strong_synergy_list)
    std_strong_synergy = np.std(strong_synergy_list)
    
    # ---- individual level ---- #
    avg_alfa_accuracy = np.mean(alfa_accuracy_list)
    std_alfa_accuracy = np.std(alfa_accuracy_list)

    avg_beta_accuracy = np.mean(beta_accuracy_list)
    std_beta_accuracy = np.std(beta_accuracy_list)

    avg_charlie_accuracy = np.mean(charlie_accuracy_list)
    std_charlie_accuracy = np.std(charlie_accuracy_list)

    avg_delta_accuracy = np.mean(delta_accuracy_list)
    std_delta_accuracy = np.std(delta_accuracy_list)

    avg_alfa_adaptation = np.mean(alfa_adaptation_list)
    avg_beta_adaptation = np.mean(beta_adaptation_list)
    avg_charlie_adaptation = np.mean(charlie_adaptation_list)
    avg_delta_adaptation = np.mean(delta_adaptation_list)
    
    ## dictionaries ## 
    exercise_run_dict_group = {
    'alfa': student_list[0], 
    'beta':student_list[1], 
    'charlie':student_list[2], 
    'delta':student_list[3], 
    'avg_accuracy': avg_group_accuracy, 
    'std_accuracy': std_group_accuracy, 
    'avg_n_tick': avg_group_n_tick, 
    'std_n_tick': std_group_n_tick, 
    'avg_consensus': avg_consensus,
    'std_consensus': std_consensus,
    'avg_weak_synergy': avg_weak_synergy,
    'std_weak_synergy': std_weak_synergy, 
    'avg_strong_synergy': avg_strong_synergy,
    'std_strong_synergy': std_strong_synergy,
    'diversity_score': diversity_score}

    exercise_run_list_ind = list(zip(student_list, 
                                    [student_list, student_list, student_list, student_list], 
                                    [avg_alfa_accuracy, avg_beta_accuracy, avg_charlie_accuracy, avg_delta_accuracy],
                                    [std_alfa_accuracy, std_beta_accuracy, std_charlie_accuracy, std_delta_accuracy], 
                                    [avg_alfa_adaptation, avg_beta_adaptation, avg_charlie_adaptation, avg_delta_adaptation]))
    
    return exercise_run_dict_group, exercise_run_list_ind

In [499]:
def abm_model(condition, max_ticks: int, n_part_exercises: int, range_solution: list, n_simulations: int):
    condition_group_df = pd.DataFrame(columns = ['alfa', 'beta', 'charlie', 'delta', 'avg_accuracy', 'std_accuracy', 'avg_n_tick', 'std_n_tick', 'avg_consensus', 'std_consensus', 'avg_weak_synergy', 'std_weak_synergy', 'avg_strong_synergy', 'std_strong_synergy', 'diversity_score'])
    condition_ind_df = pd.DataFrame(columns=['type', 'studygroup', 'avg_accuracy', 'std_accuracy', 'avg_adaptation']) #'avg_accuracy', 'contribution', 'n_presentations'])
        
    for i in (range(len(condition.index))):
        student_list = list(condition.iloc[i][1:5])
        
        ## Getting data frames for the group and individual levels ##
        exercise_run_dict_group, exercise_run_list_ind = exercise_run(student_list, max_ticks, n_part_exercises, range_solution, n_simulations) #creating dictionary
        
        exercise_run_group_df = pd.DataFrame(exercise_run_dict_group, index = [i]) #creating dataframe
        condition_group_df = pd.concat([condition_group_df, exercise_run_group_df], ignore_index = True) #concatenating dataframes
        
        exercise_run_ind_df = pd.DataFrame(exercise_run_list_ind, columns=['type', 'studygroup', 'avg_accuracy', 'std_accuracy', 'avg_adaptation']) #'avg_accuracy', 'contribution', 'n_presentations'])
        condition_ind_df = pd.concat([condition_ind_df, exercise_run_ind_df], ignore_index = True)

    return condition_ind_df, condition_group_df


In [500]:
all_ind, all_group = abm_model(condition = all_groups, 
        max_ticks = 20, 
        n_part_exercises = 8, 
        range_solution = [1, 9], 
        n_simulations = 10
        )
        

In [511]:
all_group_sorted = all_group.sort_values("avg_consensus", ascending = True)
all_group_sorted.head(50)

,alfa,beta,charlie,delta,avg_accuracy,std_accuracy,avg_n_tick,std_n_tick,avg_consensus,std_consensus,avg_weak_synergy,std_weak_synergy,avg_strong_synergy,std_strong_synergy,diversity_score
1970,ISFJ,ENTP,ISFP,ISTP,0.8625,0.087500,13.6,8.151074,0.4,0.489898,0.134375,0.079119,-0.0250,0.075000,5
1124,ESTJ,ISTJ,ESFP,ISFP,0.8625,0.087500,16.3,6.116371,0.4,0.489898,0.093750,0.076547,-0.0500,0.082916,6
1092,ESTJ,ISTJ,INFJ,ESFP,0.8875,0.067315,14.0,6.340347,0.5,0.500000,0.075000,0.101934,-0.0500,0.114564,6
2027,ISFJ,INTJ,ESFP,ESTP,0.8500,0.108972,12.8,7.858753,0.5,0.500000,0.071875,0.071329,-0.0500,0.114564,7
2314,ISTJ,INFP,ENTP,ESFP,0.9375,0.062500,12.1,8.560958,0.5,0.500000,0.121875,0.053125,0.0000,0.055902,7
1451,ESTJ,INTP,ESFP,ESFP,0.8625,0.103833,15.2,6.161169,0.5,0.500000,0.103125,0.103692,-0.0625,0.100778,5
2505,ISTJ,ESTP,ISFP,ISTP,0.8250,0.114564,16.1,5.281098,0.5,0.500000,0.084375,0.110087,-0.0750,0.114564,3
1616,ISFJ,ISTJ,ENFP,INFP,0.8625,0.117925,11.7,8.660831,0.5,0.500000,0.046875,0.074280,-0.0625,0.083853,6
580,ESFJ,ENFJ,INTP,ISTP,0.8875,0.103833,13.0,8.282512,0.5,0.500000,0.103125,0.096875,-0.0500,0.114564,8
2827,ENFP,INTP,INTJ,ESTP,0.9125,0.080039,12.1,8.239539,0.5,0.500000,0.106250,0.067315,-0.0125,0.103833,5


In [507]:
all_group.to_csv('10sim_NEW_all_group_3.csv')